In [1]:
from clinical_ts.timeseries_utils import *
from clinical_ts.ecg_utils import *
from pathlib import Path
import numpy as np

/home/zoeyhuang/miniconda3/envs/MGB-MAIDAP/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## PTB-XL

In [2]:
target_fs=100 # sampling rate (100 Hz or 500 Hz)
data_folder_ptb_xl = Path("/home/shared/physionet.org/files/ptb-xl/1.0.3")
target_folder_ptb_xl = Path("./processed_ptb_xl_fs"+str(target_fs)) 

In [3]:
# Prepare the dataset
df_ptb_xl, lbl_itos_ptb_xl,  mean_ptb_xl, std_ptb_xl = prepare_data_ptb_xl(data_folder_ptb_xl, min_cnt=0, target_fs=target_fs, channels=12, channel_stoi=channel_stoi_default, target_folder=target_folder_ptb_xl)

df_ptb_xl columns Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')
df_ptb_xl Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'label_all', 'label_diag', 'label_form', '

100%|██████████| 21799/21799 [09:33<00:00, 38.00it/s]


Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'label_all', 'label_diag', 'label_form', 'label_rhythm',
       'label_diag_subclass', 'label_diag_superclass', 'label_age',
       'label_sex', 'label_weight', 'label_height', 'dataset',
       'label_all_numeric', 'label_diag_numeric', 'label_form_numeric',
       'label_rhythm_numeric', 'label_diag_subclass_numeric',
       'label_diag_superclass_numeric', 'label_age_numeric',
       'label_sex_numeric', 'label_weight_numeric', 'label_height_numeric',
       'data'],
      dtype='object')
{'label_all': array(['1AVB', '2AVB', '3AVB', '

In [4]:
print(lbl_itos_ptb_xl.keys())

dict_keys(['label_all', 'label_diag', 'label_form', 'label_rhythm', 'label_diag_subclass', 'label_diag_superclass', 'label_age', 'label_sex', 'label_weight', 'label_height'])


In [5]:
#reformat everything as memmap for efficiency
reformat_as_memmap(df_ptb_xl, target_folder_ptb_xl/("memmap.npy"),data_folder=target_folder_ptb_xl,delete_npys=True)

100%|██████████| 21799/21799 [03:31<00:00, 103.22it/s]


Saved df to processed_ptb_xl_fs100/df_memmap.pkl


,patient_id,age,sex,height,weight,nurse,site,device,recording_date,report,...,label_diag_superclass_numeric,label_age_numeric,label_sex_numeric,label_weight_numeric,label_height_numeric,data,data_mean,data_std,data_length,data_original
ecg_id,,,,,,,,,,,,,,,,,,,,,
1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,...,[3],[3],[1],[2],[0],0,"[0.0025305834, 0.002238605, -0.001173855, 0.00...","[0.11038501, 0.085274525, 0.113332465, 0.21522...",1000,00001_hr.npy
2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,...,[3],[0],[0],[2],[0],1,"[-0.0035542173, -0.0034309644, 0.0044596153, 0...","[0.1301214, 0.24356228, 0.21888414, 0.42209703...",1000,00002_hr.npy
3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,sinusrhythmus normales ekg,...,[3],[1],[1],[2],[0],2,"[-0.0027685824, 0.0057316544, -0.00033551158, ...","[0.1212705, 0.14372103, 0.11626389, 0.14763561...",1000,00003_hr.npy
4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,sinusrhythmus normales ekg,...,[3],[0],[0],[3],[0],3,"[-0.0011030551, 0.016957743, -0.0035870918, -0...","[0.13666143, 0.4285196, 0.18045565, 0.42638788...",1000,00004_hr.npy
5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,sinusrhythmus normales ekg,...,[3],[0],[1],[2],[0],4,"[-0.0036089725, -0.0025976324, 0.001548249, 0....","[0.08276952, 0.23692654, 0.14219096, 0.3419675...",1000,00005_hr.npy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21833,17180.0,67.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-05-31 09:14:35,ventrikulÄre extrasystole(n) sinustachykardie ...,...,[4],[4],[1],[0],[0],21794,"[-0.011505826, -0.0059250575, -0.00025438584, ...","[0.21443546, 0.22212309, 0.13160513, 0.2792728...",1000,21833_hr.npy
21834,20703.0,300.0,0,NaN,NaN,1.0,2.0,AT-60 3,2001-06-05 11:33:39,sinusrhythmus lagetyp normal qrs(t) abnorm ...,...,[3],[6],[0],[0],[0],21795,"[-0.0030880228, -0.0040133577, -0.0014986034, ...","[0.09587885, 0.07901368, 0.09601028, 0.1929628...",1000,21834_hr.npy
21835,19311.0,59.0,1,NaN,NaN,1.0,2.0,AT-60 3,2001-06-08 10:30:27,sinusrhythmus lagetyp normal t abnorm in anter...,...,[4],[3],[1],[0],[0],21796,"[-0.001677597, -0.005183384, 0.0025052682, 0.0...","[0.09145833, 0.09912553, 0.12619448, 0.1590776...",1000,21835_hr.npy


In [6]:
df_ptb_xl.columns

Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'label_all', 'label_diag', 'label_form', 'label_rhythm',
       'label_diag_subclass', 'label_diag_superclass', 'label_age',
       'label_sex', 'label_weight', 'label_height', 'dataset',
       'label_all_numeric', 'label_diag_numeric', 'label_form_numeric',
       'label_rhythm_numeric', 'label_diag_subclass_numeric',
       'label_diag_superclass_numeric', 'label_age_numeric',
       'label_sex_numeric', 'label_weight_numeric', 'label_height_numeric',
       'data', 'data_mean', 'data_std', 'data_length'],
      dtype='object')

In [7]:
# print the first dataline of the dataframe
df_ptb_xl.iloc[0]

patient_id                                                                 15709.0
age                                                                           56.0
sex                                                                              1
height                                                                         NaN
weight                                                                        63.0
nurse                                                                          2.0
site                                                                           0.0
device                                                                   CS-12   E
recording_date                                                 1984-11-09 09:17:34
report                                      sinusrhythmus periphere niederspannung
scp_codes                                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
heart_axis                                                                     NaN
infa

In [8]:
# check all the columns and the datatypes
df_ptb_xl.dtypes

patient_id                       float64
age                              float64
sex                                int64
height                           float64
weight                           float64
nurse                            float64
site                             float64
device                            object
recording_date                    object
report                            object
scp_codes                         object
heart_axis                        object
infarction_stadium1               object
infarction_stadium2               object
validated_by                     float64
second_opinion                      bool
initial_autogenerated_report        bool
validated_by_human                  bool
baseline_drift                    object
static_noise                      object
burst_noise                       object
electrodes_problems               object
extra_beats                       object
pacemaker                         object
strat_fold      

## PTB-XL dataloader

In [9]:
input_size = 1000  # Sample length

chunkify_train = False
chunk_length_train = input_size if chunkify_train else 0
stride_train = input_size
        
chunkify_valtest = False
chunk_length_valtest = input_size if chunkify_valtest else 0
stride_valtest = input_size

In [10]:
df_mapped, lbl_itos,  mean, std = load_dataset(target_folder_ptb_xl)
print("lbl_itos", lbl_itos.keys())
print(df_mapped.columns)

ds_mean = np.array([-0.00184586, -0.00130277,  0.00017031, -0.00091313, -0.00148835,  -0.00174687, -0.00077071, -0.00207407,  0.00054329,  0.00155546,  -0.00114379, -0.00035649])
ds_std = np.array([0.16401004, 0.1647168 , 0.23374124, 0.33767231, 0.33362807,  0.30583013, 0.2731171 , 0.27554379, 0.17128962, 0.14030828,   0.14606956, 0.14656108])

def multihot_encode(x, num_classes):
    res = np.zeros(num_classes,dtype=np.float32)
    for y in x:
        res[y]=1
    return res

# multiple labels
all_meta_labels = ['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'label_all', 'label_diag', 'label_form', 'label_rhythm',
       'label_diag_subclass', 'label_diag_superclass', 'dataset',
       'label_all_numeric', 'label_diag_numeric', 'label_form_numeric',
       'label_rhythm_numeric', 'label_diag_subclass_numeric',
       'label_diag_superclass_numeric', 'data', 'data_mean', 'data_std',
       'data_length']

ptb_xl_label_demographcis = ["label_all", "label_age", "label_sex", "label_height", "label_weight"]
# Label all has 71 classes, label age has 7 classes, label sex has 2 classes, label height has 5 classes, label weight has 5 classes

df_mapped["label"] = df_mapped.apply(
    lambda row: np.concatenate([
        multihot_encode(row[label+"_numeric"], len(lbl_itos[label]))
        for label in ptb_xl_label_demographcis
    ]),
    axis=1
)

# print(df_mapped["label"])
print(len(df_mapped["label"].iloc[0]))
tfms_ptb_xl_cpc = ToTensor()
            
max_fold_id = df_mapped.strat_fold.max()
print(df_mapped["strat_fold"].value_counts())

df_train = df_mapped[df_mapped.strat_fold<max_fold_id-1]
df_val = df_mapped[df_mapped.strat_fold==max_fold_id-1]
df_test = df_mapped[df_mapped.strat_fold==max_fold_id]

lbl_itos dict_keys(['label_all', 'label_diag', 'label_form', 'label_rhythm', 'label_diag_subclass', 'label_diag_superclass', 'label_age', 'label_sex', 'label_weight', 'label_height'])
Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr',
       'label_all', 'label_diag', 'label_form', 'label_rhythm',
       'label_diag_subclass', 'label_diag_superclass', 'label_age',
       'label_sex', 'label_weight', 'label_height', 'dataset',
       'label_all_numeric', 'label_diag_numeric', 'label_form_numeric',
       'label_rhythm_numeric', 'label_diag_subclass_numeric',
       'label_diag_superclass_numeric

In [11]:
df_train["label"].iloc[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0.], dtype=float32)

In [12]:
# Here are the PTB-XL dataloaders

ds_train=TimeseriesDatasetCrops(df_train,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_train,min_chunk_length=input_size, stride=stride_train,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label" ,memmap_filename=target_folder_ptb_xl/("memmap.npy"))
ds_val=TimeseriesDatasetCrops(df_val,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_valtest,min_chunk_length=input_size, stride=stride_valtest,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label",memmap_filename=target_folder_ptb_xl/("memmap.npy"))
ds_test=TimeseriesDatasetCrops(df_test,input_size,num_classes=len(lbl_itos),data_folder=target_folder_ptb_xl,chunk_length=chunk_length_valtest,min_chunk_length=input_size, stride=stride_valtest,transforms=tfms_ptb_xl_cpc,annotation=False,col_lbl ="label",memmap_filename=target_folder_ptb_xl/("memmap.npy"))

In [13]:
# Save splits into npy files with data and labels
train_data_npy = []
train_label_npy = []
for i in range(len(ds_train)):
    train_data_npy.append(ds_train[i].data)
    train_label_npy.append(ds_train[i].label)
train_data_npy = np.array(train_data_npy)
trai_label_npy = np.array(train_label_npy)

np.save(target_folder_ptb_xl/"ptbxl_train_data.npy", train_data_npy)
np.save(target_folder_ptb_xl/"ptbxl_train_labels.npy", train_label_npy)

val_data_npy = []
val_label_npy = []
for i in range(len(ds_val)):
    val_data_npy.append(ds_val[i].data)
    val_label_npy.append(ds_val[i].label)
val_data_npy = np.array(val_data_npy)
val_label_npy = np.array(val_label_npy)

np.save(target_folder_ptb_xl/"ptbxl_val_data.npy", val_data_npy)
np.save(target_folder_ptb_xl/"ptbxl_val_labels.npy", val_label_npy)


In [14]:
test_data_npy = []
test_label_npy = []
for i in range(len(ds_test)):
    test_data_npy.append(ds_test[i].data)
    test_label_npy.append(ds_test[i].label)
test_data_npy = np.array(test_data_npy)
test_label_npy = np.array(test_label_npy)

np.save(target_folder_ptb_xl/"ptbxl_test_data.npy", test_data_npy)
np.save(target_folder_ptb_xl/"ptbxl_test_labels.npy", test_label_npy)

In [15]:
# Load and check the shape of the saved npy files
train_data = np.load(target_folder_ptb_xl/"ptbxl_train_data.npy")
train_labels = np.load(target_folder_ptb_xl/"ptbxl_train_labels.npy")
print(train_data.shape)
print(train_labels.shape)

(17418, 12, 1000)
(17418, 90)
